In [1]:
# الخلية 1: الاستيرادات والإعدادات
import pandas as pd
import chromadb
from sentence_transformers import SentenceTransformer
import os
import shutil # لمسح المجلد القديم إذا تطلب الأمر

# ==========================================
# إعدادات المشروع
# ==========================================
# 1. مسار ملف البيانات النظيف (تأكد من وجوده بجانب هذا الملف)
INPUT_CSV = 'medical_data_cleaned.csv'

# 2. مسار مجلد قاعدة البيانات (سيتم إنشاؤه تلقائياً)
DB_PATH = "./chroma_db_storage"

# 3. اسم المجموعة (Collection) داخل القاعدة
COLLECTION_NAME = "medical_knowledge_base"

# 4. اسم نموذج التضمين (يدعم العربية بقوة)
# هذا النموذج خفيف (420MB تقريباً) وسريع ودقيق
MODEL_NAME = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'

print("✅ تم إعداد المسارات بنجاح.")

/home/ahmed/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ تم إعداد المسارات بنجاح.


In [2]:
# الخلية 2: تحميل الموديل وقراءة ملف الـ CSV
try:
    # 1. تحميل نموذج الـ Embeddings
    print(f"⏳ جاري تحميل نموذج الذكاء الاصطناعي: {MODEL_NAME}...")
    embedding_model = SentenceTransformer(MODEL_NAME)
    print("✅ تم تحميل النموذج بنجاح.")

    # 2. قراءة ملف البيانات
    print(f"⏳ جاري قراءة الملف: {INPUT_CSV}...")
    df = pd.read_csv(INPUT_CSV, encoding='utf-8-sig')
    
    # التأكد من وجود الأعمدة المطلوبة
    if 'cleaned_question' not in df.columns or 'cleaned_answer' not in df.columns:
        raise ValueError("❌ الملف لا يحتوي على الأعمدة المطلوبة: cleaned_question, cleaned_answer")
        
    print(f"✅ تم تحميل البيانات: {len(df)} سجل.")
    
except Exception as e:
    print(f"❌ حدث خطأ: {e}")

⏳ جاري تحميل نموذج الذكاء الاصطناعي: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2...
✅ تم تحميل النموذج بنجاح.
⏳ جاري قراءة الملف: medical_data_cleaned.csv...
✅ تم تحميل البيانات: 808471 سجل.


In [3]:
# الخلية 3: تهيئة قاعدة البيانات
# تنبيه: هذا الكود يقوم بإنشاء قاعدة جديدة، إذا كانت موجودة سيقوم بحذفها وإعادة إنشائها لضمان التحديث

print("⚙️ جاري إعداد قاعدة البيانات...")

# استخدام PersistentClient للحفظ الدائم على القرص
client = chromadb.PersistentClient(path=DB_PATH)

# التحقق مما إذا كانت المجموعة موجودة وحذفها (لبدء نظيف)
try:
    client.delete_collection(name=COLLECTION_NAME)
    print(f"   - تم حذف المجموعة القديمة '{COLLECTION_NAME}' لتحديث البيانات.")
except:
    print(f"   - جاري إنشاء مجموعة جديدة '{COLLECTION_NAME}'...")

# إنشاء المجموعة (المكان الذي نخزن فيه البيانات)
collection = client.create_collection(name=COLLECTION_NAME)
print("✅ قاعدة البيانات جاهزة لاستقبال البيانات.")

⚙️ جاري إعداد قاعدة البيانات...
   - جاري إنشاء مجموعة جديدة 'medical_knowledge_base'...
✅ قاعدة البيانات جاهزة لاستقبال البيانات.


In [4]:
# الخلية 4: التضمين والتخزين (قد تأخذ وقتاً حسب حجم البيانات)

print("🚀 بدء عملية الفهرسة (Indexing)...")

# قوائم مؤقتة لتجميع البيانات
ids = []
documents = []
embeddings = []
metadatas = []

# حجم الدفعة (يفضل معالجتها على دفعات لعدم استهلاك الذاكرة)
BATCH_SIZE = 64 

total_rows = len(df)

for i in range(0, total_rows, BATCH_SIZE):
    # تحديد بداية ونهاية الدفعة
    end_i = min(i + BATCH_SIZE, total_rows)
    batch_df = df.iloc[i:end_i]
    
    # تجهيز الدفعة الحالية
    batch_ids = [str(idx) for idx in batch_df.index]
    batch_docs = batch_df['cleaned_question'].tolist() # سنبحث باستخدام الأسئلة
    
    # حساب الـ Embeddings باستخدام النموذج
    batch_embeddings = embedding_model.encode(batch_docs).tolist()
    
    # تجهيز البيانات الوصفية (الجواب + التصنيف)
    batch_metadatas = []
    for _, row in batch_df.iterrows():
        meta = {
            "answer": row['cleaned_answer'],
            "original_question": row['Question'],
            "category": str(row['Category']) if 'Category' in row else "General"
        }
        batch_metadatas.append(meta)
    
    # الحفظ في ChromaDB
    collection.add(
        ids=batch_ids,
        documents=batch_docs,
        embeddings=batch_embeddings,
        metadatas=batch_metadatas
    )
    
    print(f"   - تمت معالجة الدفعة: {i} إلى {end_i} من أصل {total_rows}")

print("-" * 50)
print(f"🎉 تم الانتهاء! تم حفظ قاعدة البيانات في المجلد: {DB_PATH}")
print("يمكنك الآن إغلاق هذا الملف والانتقال لملف الاختبار.")

🚀 بدء عملية الفهرسة (Indexing)...
   - تمت معالجة الدفعة: 0 إلى 64 من أصل 808471
   - تمت معالجة الدفعة: 64 إلى 128 من أصل 808471
   - تمت معالجة الدفعة: 128 إلى 192 من أصل 808471
   - تمت معالجة الدفعة: 192 إلى 256 من أصل 808471
   - تمت معالجة الدفعة: 256 إلى 320 من أصل 808471
   - تمت معالجة الدفعة: 320 إلى 384 من أصل 808471
   - تمت معالجة الدفعة: 384 إلى 448 من أصل 808471
   - تمت معالجة الدفعة: 448 إلى 512 من أصل 808471
   - تمت معالجة الدفعة: 512 إلى 576 من أصل 808471
   - تمت معالجة الدفعة: 576 إلى 640 من أصل 808471
   - تمت معالجة الدفعة: 640 إلى 704 من أصل 808471
   - تمت معالجة الدفعة: 704 إلى 768 من أصل 808471
   - تمت معالجة الدفعة: 768 إلى 832 من أصل 808471
   - تمت معالجة الدفعة: 832 إلى 896 من أصل 808471
   - تمت معالجة الدفعة: 896 إلى 960 من أصل 808471
   - تمت معالجة الدفعة: 960 إلى 1024 من أصل 808471
   - تمت معالجة الدفعة: 1024 إلى 1088 من أصل 808471
   - تمت معالجة الدفعة: 1088 إلى 1152 من أصل 808471
   - تمت معالجة الدفعة: 1152 إلى 1216 من أصل 808471
   - تمت معال